In [1]:
import paragami
import autograd
import autograd.numpy as np

In [2]:
pattern = paragami.NumericArrayPattern(shape=(2, 3), lb=0, ub=10)
x = pattern.empty(valid=False)
x_bool = np.full_like(x, False, dtype='bool')
print(x_bool)
x_bool[1, 2] = True
x_bool[1, 1] = True
pattern.flat_indices(x_bool, free=True)

[[False False False]
 [False False False]]


array([4, 5])

In [3]:
pattern.flatten(x_bool, free=False)

array([False, False, False, False,  True,  True])

In [19]:
# it = np.nditer(x_bool, flags=['multi_index'])
# while not it.finished:
#     print("%s <%s>" % (it[0], it.multi_index), end=' ')
#     it.iternext()

import itertools
free_len = pattern.flat_length(free=True)
flat_len = pattern.flat_length(free=False)
manual_jac = np.zeros((free_len, flat_len))
x_bool = np.full_like(x, False, dtype='bool')
for ind in itertools.product(*(range(n) for n in x_bool.shape)):
    x_bool[ind] = True
    flat_ind = pattern.flat_indices(x_bool, free=False)
    free_ind = pattern.flat_indices(x_bool, free=True)
    manual_jac[np.ix_(flat_ind, free_ind)] = 1
    x_bool[ind] = False

print(manual_jac)

def flat_to_free(flat_val):
    return pattern.flatten(pattern.fold(flat_val, free=False), free=True)

flat_to_free(np.ones(flat_len))

flat_to_free_jac = autograd.jacobian(flat_to_free)
print(flat_to_free_jac(np.ones(flat_len)) != 0)
print(manual_jac != 0)
print(np.all((manual_jac != 0) == (flat_to_free_jac(np.ones(flat_len)) != 0)))


[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ True False False False False False]
 [False  True False False False False]
 [False False  True False False False]
 [False False False  True False False]
 [False False False False  True False]
 [False False False False False  True]]
[[ True False False False False False]
 [False  True False False False False]
 [False False  True False False False]
 [False False False  True False False]
 [False False False False  True False]
 [False False False False False  True]]
True
